In [1]:
%config IPCompleter.greedy=True

import torch
import os
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import itertools
import math
import pandas as pd
import unicodedata
import codecs
import itertools


from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import numpy as np

import  pprint
pp = pprint.PrettyPrinter(indent = 4)

In [2]:
CUDA = torch.cuda.is_available()
device = torch.device("cuda" if CUDA else "cpu")

# DATA


leave out validation set

In [3]:
INPUT_SIZE = 2000
TOKENS_TAKEN = 0

xy = pd.read_csv('./prep_student_input.csv')


randomindexes = [random.randint(0, INPUT_SIZE) for _ in range(INPUT_SIZE)]


pastconvo = [xy['Past Convo'][randomindexes[i]] for i in range(len(randomindexes))]
pastdf = pd.DataFrame(data=pastconvo)

tutor = [xy['Tutor Response'][randomindexes[i]] for i in range(len(randomindexes))]
tutordf = pd.DataFrame(data=tutor)



pastconvo = pastdf.values.flatten().tolist()

response = tutordf.values.flatten().tolist()




pairs = [[' '.join(pastconvo[i].split()[TOKENS_TAKEN:]), response[i]] for i in range(len(pastconvo))]


print("input data selected")

input data selected


# VOCABUOLARY


In [4]:
# Default word tokens
PAD_token = 0  # Used for padding short sentences
SOS_token = 1  # Start-of-sentence token
EOS_token = 2  # End-of-sentence token

class Vocabulary:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3  # Count SOS, EOS, PAD

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # Remove words below a certain count threshold # CHANGE probably shouldn't do this
    def trim(self, min_count):
        keep_words = []
        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)        
        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) / len(self.word2index)
        ))
        # Reinitialize dictionaries
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens

        for word in keep_words:
            self.addWord(word)

In [5]:
# make data simple
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

In [6]:
# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# creating vocab


In [7]:
pairs = [[normalizeString(line) for line in p] for p in pairs]

pastwhole = pd.DataFrame(data=xy['Past Convo']).values.flatten().tolist()
tutorwhole = pd.DataFrame(data=xy['Tutor Response']).values.flatten().tolist()

pairs_whole = [[pastwhole[i], tutorwhole[i]] for i in range(len(pastwhole))]

pairs_whole = [[normalizeString(line) for line in p] for p in pairs_whole]

voc = Vocabulary("dialog")


#getting  rid of rare words
for pair in pairs_whole:
    voc.addSentence(pair[0])
    voc.addSentence(pair[1])
print("counted words:", voc.num_words)
print("vocabulary created")


counted words: 1014
vocabulary created


# prepare data

In [8]:
def indexesFromSentence(voc, sentence):
    return [voc.word2index[word] for word in sentence.split(' ')] + [EOS_token]


In [9]:
#make column consistent and transpose
def zeroPadding(l, fillvalue= 0):
    return list(itertools.zip_longest(*l, fillvalue=fillvalue))

In [10]:
def binaryMatrix(l, value=0):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == PAD_token:
                m[i].append(0)
            else:
                m[i].append(1)
    return m

In [11]:
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch)
    mask = binaryMatrix(padList)
    mask = torch.ByteTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len, input_batch, output_batch

# MODEL

In [12]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

Attention using dot

In [13]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        self.hidden_size = hidden_size

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        attn_energies = self.dot_score(hidden, encoder_outputs)
        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()
        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [14]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

# Training

In [15]:
# negative log lokelihood loss
# maskNLLLoss(decoder_output, target_variable[t], mask[t])
def maskNLLLoss(inp, target, mask): 
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    #loss = reward * loss -> 0.001/1
    loss = loss.to(device)
    return loss, nTotal.item()

In [16]:
LEARN_RATE = 0.001
clip = 50.0
# num_iters = 400
DESIRED_LOSS = 0.2

small_batch_size = 20
batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len, input_batch, output_batch = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

attn_model = 'dot'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
embedding = nn.Embedding(voc.num_words, hidden_size)

encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model,  embedding, hidden_size, voc.num_words, decoder_n_layers, dropout)
#pass models to device 
encoder = encoder.to(device)
decoder = decoder.to(device)

encoder.train()
decoder.train()

#optimizers
encoder_optimizer = optim.Adam(encoder.parameters(), lr=LEARN_RATE)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=LEARN_RATE)
encoder_optimizer.zero_grad()
decoder_optimizer.zero_grad()


# start training
print("START TRAINING")
print("\n")
returned_loss = 1000


iteration = 0
# for iteration in range(num_iters):
while returned_loss > DESIRED_LOSS:
    iteration += 1
    
    print(" Iteration:", iteration)
    
    batches = batch2TrainData(voc, [random.choice(pairs) for _ in range(small_batch_size)])
    input_variable, lengths, target_variable, mask, max_target_len, input_batch, output_batch = batches
    
    #train one iter
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    loss = 0
    print_losses = []
    n_totals = 0

    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)
#     print("encoder output shape", encoder_outputs.shape)
#     print("last encoder hidden shape", encoder_hidden.shape)

    decoder_input = torch.LongTensor([[SOS_token for _ in range (small_batch_size)]])
    decoder_input = decoder_input.to(device)
#     print("initial decoder input shape", decoder_input.shape)
#     print(decoder_input)

    decoder_hidden =  encoder_hidden[:decoder.n_layers]
#     print("initial decoder hidden state shape", decoder_hidden.shape)

    for t in range(max_target_len):
        decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden, encoder_outputs)

        #teacher forcing
        decoder_input = target_variable[t].view(1,-1)

        mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
        loss += mask_loss
        print_losses.append(mask_loss.item() * nTotal)

        n_totals+= nTotal

    loss.backward(retain_graph=True)
    
    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    encoder_optimizer.step()
    decoder_optimizer.step()

    returned_loss = sum(print_losses) / n_totals
    print("returned loss", returned_loss)
    print("\n")
    



KeyError: 'Tutor:'

In [20]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores
    

In [21]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc, input_sentence, max_length):
    input_sentence = normalizeString(input_sentence)
    # Evaluate sentence
    output_words = evaluate(encoder, decoder, searcher, voc, input_sentence, max_length)
    # Format and print response sentence
    output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
    print('Bot:', ' '.join(output_words))


In [70]:
#evaluate model
TEST_SIZE = 30

encoder.eval()
decoder.eval()

searcher = GreedySearchDecoder(encoder, decoder)

randomindexes = [random.randint(INPUT_SIZE, len(xy['Past Convo'])) for _ in range(TEST_SIZE)]
inputs = [xy['Past Convo'][randomindexes[i]] for i in range(TEST_SIZE)]
outputs = [xy['Tutor Response'][randomindexes[i]] for i in range(TEST_SIZE)]

In [81]:
for i in range(small_batch_size):
    input_sentence = inputs[i]
    max_length = len(normalizeString(outputs[i]).split()) 
    print("past convo:", input_sentence)
    print("\n")
    evaluateInput(encoder, decoder, searcher, voc, input_sentence, max_target_len)
    print("\n")
    print("Correct: ", outputs[i])
    print("\n")

past convo: Tutor: "Pink" is "rosa". Please try to fill in the blank in Italian.  Student: Okay, what is the word for box again?


Bot: the word for box is scatola . remember that adjectives follow the noun they modify ? try again . try to fill


Correct:  The word for box is "scatola".


past convo: Tutor: Please try to fill in the blank in Italian.  Student: what is behind in italian?


Bot: e dietro la repeat to yourself several times


Correct:  Behind is dietro.


past convo: Tutor: "Is Inside Of The" is "e dentro la". Please try to fill in the blank in Italian.  Student: What is yellow box in Italian? Tutor: Okay, I'll give you a hint.  "box" is  "scatola" Student: What is yellow in Italian?


Bot: yellow is gialla notice how tell you say the dog now ? write the color of the table to your sentence ?


Correct:  "Yellow" is "gialla": gialla is the feminine form, because the noun it modifies (scatola) is feminine.


past convo: Tutor: Please try to fill in the blank in Italian.  St

# simulated student

In [35]:
import  pprint
pp = pprint.PrettyPrinter(indent = 4)


xy = pd.read_csv('./prep_student_input.csv')
prepdf = pd.DataFrame(data=xy['Prep'])
colordf = pd.DataFrame(data=xy['Color'])
objdf = pd.DataFrame(data=xy['Obj'])


prep = list(set(prepdf.values.flatten().tolist()))
color = list(set(colordf.values.flatten().tolist()))
obj = list(set(objdf.values.flatten().tolist()))

concepts = prep_u + color_u + obj_u


#this is type set
knowledgeA = {concept: 0 for concept in concepts}

# pp.pprint(concepts)

studentA = {
    "pickup" : 1,
    "forget" : 0,
    "knowledge" : knowledgeA
}
    

In [40]:
#for each question
prep_confidence = {concept: 0 for concept in prep}
color_confidence = {concept: 0 for concept in color}
obj_confidence = {concept: 0 for concept in obj}

confidence = {
    "prep": prep_confidence,
    "color": color_confidence,
    "obj": obj_confidence
}

studentA["confidence"]  = confidence

In [2]:
sample = "pink is rosa"

#better way to pick out comcepts
def pickout_concepts(resp):
    #return a list of concepts present in  resp
    rv = []
    for word in resp.split():
        if word in concepts:
            rv.append(word)
    return rv
            
#use pickout rate and forget rate
def knowledge_change(resp):
    picked = pickout_concepts(resp)
    
    change = 0
    for c in picked:
        if studentA["knowledge"][c] ==  0:
            if random.random() < studentA["pickup"]: 
                studentA["knowledge"][c] = 1
                change += 1
        elif studentA["knowledge"][c] ==  1:
            if random.random() < studentA["forget"]: 
                studentA["knowledge"][c] = 0
                change -= 1
    return change

# print(pickout_concepts(sample))

def simulate(encoder, decoder, searcher, voc):
    input_sentence = ''
    learned = 0
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = normalizeString(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence, max_target_len)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            output = ' '.join(output_words)
            
            change = knowledge_change(output)
            learned += change
            
            print('Bot:', output)
            print('change:', change)
            print('knowledge', learned)

        except KeyError:
            print("Error: Encountered unknown word.")
            

In [3]:
searcher = GreedySearchDecoder(encoder, decoder)

simulate(encoder, decoder, searcher, voc)

NameError: name 'GreedySearchDecoder' is not defined

In [ ]:
# # student only guess (3 obj)
# # ask question if  confidence low

# is loss same in 100 steps, halt train
# in training loop